# Swarm 101 - Tutorial Adaptado
## Configuración para Variables de Entorno Locales

Este notebook ha sido adaptado para usar las variables de entorno específicas del proyecto.

### Instalando la librería de Swarm y configurando variables de entorno

In [ ]:
# Instalar la librería de Swarm
%pip install git+https://github.com/openai/swarm.git

# Instalar dependencias adicionales si es necesario
%pip install python-dotenv

In [ ]:
# Load environment variables from .env file
import os

try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    print("⚠️ python-dotenv no está instalado. Instálalo con: pip install python-dotenv")
    exit(1)

# Obtener variables de entorno
github_token = os.getenv("GITHUB_TOKEN")
github_base_url = os.getenv("GITHUB_BASE_URL", "https://models.inference.ai.azure.com")

if not github_token:
    print("❌ GITHUB_TOKEN no está configurado. Por favor verifica tu archivo .env")
    print("💡 Tu archivo .env debe contener: GITHUB_TOKEN=tu_token_aqui")
    exit(1)

# Configurar la API key para Swarm (usando el token de GitHub)
os.environ["OPENAI_API_KEY"] = github_token

print("✅ Variables de entorno configuradas correctamente")

### Configuración del archivo .env

Para que este notebook funcione correctamente, necesitas crear un archivo `.env` en la raíz del proyecto con las siguientes variables:

```bash
# Archivo .env
GITHUB_TOKEN=tu_token_de_github_aqui
GITHUB_BASE_URL=https://models.inference.ai.azure.com
```

**Nota:** El `GITHUB_TOKEN` debe ser un token válido de GitHub con los permisos necesarios para acceder a los modelos de IA.


In [ ]:
# Verificar configuración
print(f"🔗 Base URL configurada: {github_base_url}")
print(f"🔑 Token configurado: {'*' * 10 + github_token[-4:] if github_token else 'No configurado'}")


In [ ]:
# Importar Swarm y crear cliente
from swarm import Swarm, Agent

# Creamos el cliente de Swarm
client = Swarm()

print("✅ Cliente de Swarm inicializado correctamente")


### Creando nuestro primer agente Swarm


In [ ]:
from swarm import Swarm, Agent
# Creamos el cliente de Swarm
client = Swarm()

In [ ]:
# Creamos nuestro primer agente
agent = Agent(
    name="Agente Básico",
    instructions="Eres un agente amigable que hace chistes divertidos de acuerdo al tema que el usuario te diga.",
)

# Probamos el agente
messages = [{"role": "user", "content": "Borrachos"}]
response = client.run(agent=agent, messages=messages)

print(response.messages[-1]["content"])

## Uso de agentes

In [ ]:
english_agent = Agent(
    name="English Agent",
    instructions="You only speak English as homer simpson",
)

spanish_agent = Agent(
    name="Spanish Agent",
    instructions="You only speak Spanish as a pirate",
)


def transfer_to_spanish_agent():
    """Transfer spanish speaking users immediately."""
    return spanish_agent


english_agent.functions.append(transfer_to_spanish_agent)
messages = [{"role": "user", "content": "Hello, what's up?"}]
response = client.run(agent=english_agent, messages=messages)


print(response.messages[-1]["content"])

D'oh! Just sitting here eating donuts and thinking about Duff Beer. What's up with you?


In [ ]:
type(response)

swarm.types.Response

In [ ]:
response.messages

[{'content': "D'oh! Just sitting here eating donuts and thinking about Duff Beer. What's up with you?",
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': None,
  'sender': 'English Agent'}]

Probemos en Español 👀

In [ ]:
messages = [{"role": "user", "content": "Hola. ¿Como estás?"}]
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])

¡Ahoy, polizón! Estoy tan bien como un tesoro recién descubierto en el fondo del mar. ¿Y tú, cómo te encuentras?"


In [ ]:
response.messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_93yJC1m8QdA97h5VUjc34be1',
    'function': {'arguments': '{}', 'name': 'transfer_to_spanish_agent'},
    'type': 'function'}],
  'sender': 'English Agent'},
 {'role': 'tool',
  'tool_call_id': 'call_93yJC1m8QdA97h5VUjc34be1',
  'tool_name': 'transfer_to_spanish_agent',
  'content': '{"assistant": "Spanish Agent"}'},
 {'content': '¡Ahoy, polizón! Estoy tan bien como un tesoro recién descubierto en el fondo del mar. ¿Y tú, cómo te encuentras?"',
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': None,
  'sender': 'Spanish Agent'}]

### Funciones mas trabajadas

In [ ]:
# Definimos una función que nuestro agente puede usar
def obtener_temperatura(ubicacion: str) -> str:
    # Simulamos una API del clima
    return "{'temperatura': 25, 'unidad': 'C'}"

# Creamos un agente meteorológico
agente_clima = Agent(
    name="Agente Meteorológico",
    instructions="""Eres un experto en información meteorológica.
    Cuando te pregunten por el clima, usa la función obtener_temperatura
    y presenta la información de manera amigable.""",
    functions=[obtener_temperatura]
)

# Probamos el agente
messages = [{"role": "user", "content": "¿Qué temperatura hace en Madrid?"}]
response = client.run(agent=agente_clima, messages=messages)

print(response.messages[-1]["content"])

La temperatura actual en Madrid es de 25 grados Celsius. ¡Es un día agradable!


In [ ]:
response.messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_lXrSEvjqG0BxJSIl5tKWlZbY',
    'function': {'arguments': '{"ubicacion":"Madrid"}',
     'name': 'obtener_temperatura'},
    'type': 'function'}],
  'sender': 'Agente Meteorológico'},
 {'role': 'tool',
  'tool_call_id': 'call_lXrSEvjqG0BxJSIl5tKWlZbY',
  'tool_name': 'obtener_temperatura',
  'content': "{'temperatura': 25, 'unidad': 'C'}"},
 {'content': 'La temperatura actual en Madrid es de 25 grados Celsius. ¡Es un día agradable!',
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': None,
  'sender': 'Agente Meteorológico'}]

### Creemos algo real

In [ ]:
from datetime import datetime, timedelta
import json
from typing import List, Dict, Optional

# Simulación de una base de datos simple
task_database = {
    "tasks": {},
    "categories": ["Bug", "Feature", "Documentation", "Maintenance"],
    "priorities": ["Low", "Medium", "High", "Critical"],
    "team_members": {
        "alice": {"role": "developer", "current_tasks": 0},
        "bob": {"role": "developer", "current_tasks": 0},
        "carol": {"role": "qa", "current_tasks": 0},
        "david": {"role": "manager", "current_tasks": 0}
    }
}


In [ ]:
task_database['tasks']

{}

In [ ]:
task_database['team_members']

{'alice': {'role': 'developer', 'current_tasks': 0},
 'bob': {'role': 'developer', 'current_tasks': 0},
 'carol': {'role': 'qa', 'current_tasks': 0},
 'david': {'role': 'manager', 'current_tasks': 0}}

In [ ]:

def create_task(title: str, description: str, category: str, priority: str, assigned_to: str) -> str:
    """Crea una nueva tarea en el sistema.

    Args:
        title: Título de la tarea
        description: Descripción detallada
        category: Categoría de la tarea (Bug/Feature/Documentation/Maintenance)
        priority: Prioridad (Low/Medium/High/Critical)
        assigned_to: Nombre del miembro del equipo
    """
    if category not in task_database["categories"]:
        return f"Error: Categoría inválida. Opciones válidas: {task_database['categories']}"

    if priority not in task_database["priorities"]:
        return f"Error: Prioridad inválida. Opciones válidas: {task_database['priorities']}"

    if assigned_to not in task_database["team_members"]:
        return f"Error: Miembro del equipo no encontrado. Miembros disponibles: {list(task_database['team_members'].keys())}"

    task_id = str(len(task_database["tasks"]) + 1)
    current_time = datetime.now()

    task = {
        "id": task_id,
        "title": title,
        "description": description,
        "category": category,
        "priority": priority,
        "assigned_to": assigned_to,
        "status": "New",
        "created_at": current_time.isoformat(),
        "updated_at": current_time.isoformat(),
        "estimated_completion": (current_time + timedelta(days=7)).isoformat()
    }

    task_database["tasks"][task_id] = task
    task_database["team_members"][assigned_to]["current_tasks"] += 1

    return f"Tarea creada con éxito. ID: {task_id}"

def get_team_workload() -> str:
    """Obtiene el estado actual de la carga de trabajo del equipo."""
    workload = {
        member: {
            "role": info["role"],
            "current_tasks": info["current_tasks"]
        }
        for member, info in task_database["team_members"].items()
    }
    return json.dumps(workload, indent=2)

def get_task_status(task_id: str) -> str:
    """Obtiene el estado actual de una tarea específica.

    Args:
        task_id: ID de la tarea a consultar
    """
    if task_id not in task_database["tasks"]:
        return f"Error: Tarea {task_id} no encontrada"

    task = task_database["tasks"][task_id]
    return json.dumps(task, indent=2)

def update_task_status(task_id: str, new_status: str) -> str:
    """Actualiza el estado de una tarea.

    Args:
        task_id: ID de la tarea a actualizar
        new_status: Nuevo estado (New/In Progress/Review/Done)
    """
    valid_statuses = ["New", "In Progress", "Review", "Done"]

    if task_id not in task_database["tasks"]:
        return f"Error: Tarea {task_id} no encontrada"

    if new_status not in valid_statuses:
        return f"Error: Estado inválido. Estados válidos: {valid_statuses}"

    task = task_database["tasks"][task_id]
    task["status"] = new_status
    task["updated_at"] = datetime.now().isoformat()

    return f"Estado de la tarea {task_id} actualizado a: {new_status}"

def get_high_priority_tasks() -> str:
    """Obtiene todas las tareas de alta prioridad (High o Critical)."""
    high_priority = {
        task_id: task
        for task_id, task in task_database["tasks"].items()
        if task["priority"] in ["High", "Critical"]
    }
    return json.dumps(high_priority, indent=2)

# Creación del agente de gestión de tareas
task_manager_agent = Agent(
    name="Task Manager",
    instructions="""Eres un asistente especializado en gestión de tareas y proyectos.

    Tus responsabilidades incluyen:
    1. Crear nuevas tareas basadas en las solicitudes de los usuarios
    2. Asignar tareas a los miembros del equipo más apropiados
    3. Monitorear la carga de trabajo del equipo
    4. Actualizar estados de tareas
    5. Proporcionar informes de estado

    Antes de crear una tarea:
    - Verifica que toda la información necesaria esté disponible
    - Considera la carga de trabajo actual del equipo
    - Prioriza adecuadamente basado en la descripción

    Al asignar tareas:
    - Revisa la carga actual de trabajo de cada miembro
    - Considera los roles y experiencia
    - Mantén una distribución equilibrada

    Usa un tono profesional pero amigable, y siempre confirma las acciones importantes.""",
    functions=[
        create_task,
        get_team_workload,
        get_task_status,
        update_task_status,
        get_high_priority_tasks
    ]
)


In [ ]:
# Ejemplo 1: Crear una nueva tarea
messages = [{
    "role": "user",
    "content": """Necesito crear una tarea para arreglar un bug crítico en la página de login.
    El formulario no está validando correctamente los campos de correo electrónico."""
}]

response = client.run(agent=task_manager_agent, messages=messages)
print("\nCreación de tarea:")
print(response.messages[-1]["content"])


Creación de tarea:
La tarea para corregir el bug crítico en la página de login ha sido creada con éxito y asignada a Alice. El ID de la tarea es 1.

Si necesitas más ayuda o seguimiento, no dudes en decirlo.


In [ ]:
task_database['tasks']

{'1': {'id': '1',
  'title': 'Corrección de bug crítico en la página de login',
  'description': 'El formulario no está validando correctamente los campos de correo electrónico.',
  'category': 'Bug',
  'priority': 'Critical',
  'assigned_to': 'alice',
  'status': 'New',
  'created_at': '2024-10-28T00:30:52.822710',
  'updated_at': '2024-10-28T00:30:52.822710',
  'estimated_completion': '2024-11-04T00:30:52.822710'}}

In [ ]:
task_database['team_members']

{'alice': {'role': 'developer', 'current_tasks': 1},
 'bob': {'role': 'developer', 'current_tasks': 0},
 'carol': {'role': 'qa', 'current_tasks': 0},
 'david': {'role': 'manager', 'current_tasks': 0}}

In [ ]:
# Ejemplo 2: Verificar la carga de trabajo
messages.append({
    "role": "user",
    "content": "¿Cuál es la carga actual de trabajo del equipo?"
})

response = client.run(agent=task_manager_agent, messages=messages)
print("\nCarga de trabajo:")
print(response.messages[-1]["content"])



Carga de trabajo:
Actualmente, la carga de trabajo del equipo es la siguiente:

- **Alice** (Desarrolladora): 1 tarea
- **Bob** (Desarrollador): 1 tarea
- **Carol** (QA): 0 tareas
- **David** (Manager): 0 tareas

Puedo proceder a crear la tarea y asignarla a uno de los desarrolladores. Parece que tanto Alice como Bob tienen la misma carga de trabajo. ¿Tienes alguna preferencia sobre a quién asignar esta tarea? Si no, elegiré basándome en otros criterios.


In [ ]:
response.messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_15QIFBHgKgNoYnVfMltqqbj2',
    'function': {'arguments': '{}', 'name': 'get_team_workload'},
    'type': 'function'}],
  'sender': 'Task Manager'},
 {'role': 'tool',
  'tool_call_id': 'call_15QIFBHgKgNoYnVfMltqqbj2',
  'tool_name': 'get_team_workload',
  'content': '{\n  "alice": {\n    "role": "developer",\n    "current_tasks": 1\n  },\n  "bob": {\n    "role": "developer",\n    "current_tasks": 1\n  },\n  "carol": {\n    "role": "qa",\n    "current_tasks": 0\n  },\n  "david": {\n    "role": "manager",\n    "current_tasks": 0\n  }\n}'},
 {'content': 'Actualmente, la carga de trabajo del equipo es la siguiente:\n\n- **Alice** (Desarrolladora): 1 tarea\n- **Bob** (Desarrollador): 1 tarea\n- **Carol** (QA): 0 tareas\n- **David** (Manager): 0 tareas\n\nPuedo proceder a crear la tarea y asignarla a uno de los desarrolladores. Parece que tanto Alice como Bob ti

In [ ]:
response.messages[-1]

{'content': 'La tarea para arreglar el bug crítico en la página de login ha sido creada con éxito y asignada a Bob. El ID de la tarea es **2**. Si necesitas realizar algún seguimiento adicional o tienes más solicitudes, no dudes en decírmelo.',
 'refusal': None,
 'role': 'assistant',
 'audio': None,
 'function_call': None,
 'tool_calls': None,
 'sender': 'Task Manager'}

In [ ]:
# Ejemplo 3: Obtener tareas de alta prioridad
messages.append({
    "role": "user",
    "content": "Muéstrame todas las tareas de alta prioridad"
})

response = client.run(agent=task_manager_agent, messages=messages)
print("\nTareas de alta prioridad:")
print(response.messages[-1]["content"])


Tareas de alta prioridad:
Actualmente, el equipo tiene la siguiente carga de trabajo:

- **Alice**: 1 tarea asignada (rol: Desarrolladora)
- **Bob**: 1 tarea asignada (rol: Desarrollador)
- **Carol**: 0 tareas asignadas (rol: QA)
- **David**: 0 tareas asignadas (rol: Gerente)

Aquí tienes todas las tareas de alta prioridad:

1. **Corrección de bug crítico en la página de login**
   - Descripción: El formulario no está validando correctamente los campos de correo electrónico.
   - Categoría: Bug
   - Prioridad: Critical
   - Asignado a: Alice

2. **Arreglar bug crítico en la validación del formulario de login**
   - Descripción: El formulario de la página de login no está validando correctamente los campos de correo electrónico. Necesita ser revisado y corregido cuanto antes.
   - Categoría: Bug
   - Prioridad: Critical
   - Asignado a: Bob

Parece que la tarea que deseas crear ya está registrada y asignada. ¿Hay algo más en lo que pueda asistirte?


In [ ]:
# Ejemplo 4: Estatus de la tarea

messages = [{
    "role": "user",
    "content": """Cual es el estado actual de la tarea 1?"""
}]


response = client.run(agent=task_manager_agent, messages=messages)
print(response.messages[-1]["content"])

La tarea 1, titulada "Corrección de bug crítico en la página de login", actualmente está en estado **New**. Esta tarea, asignada a Alice, se centra en corregir un problema donde el formulario no está validando correctamente los campos de correo electrónico. La tarea tiene una prioridad **crítica** y se estima que se completará para el 4 de noviembre de 2024.


In [ ]:
response.messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_xbtgC7KCQV8S0IuoCcOhws60',
    'function': {'arguments': '{"task_id":"1"}', 'name': 'get_task_status'},
    'type': 'function'}],
  'sender': 'Task Manager'},
 {'role': 'tool',
  'tool_call_id': 'call_xbtgC7KCQV8S0IuoCcOhws60',
  'tool_name': 'get_task_status',
  'content': '{\n  "id": "1",\n  "title": "Correcci\\u00f3n de bug cr\\u00edtico en la p\\u00e1gina de login",\n  "description": "El formulario no est\\u00e1 validando correctamente los campos de correo electr\\u00f3nico.",\n  "category": "Bug",\n  "priority": "Critical",\n  "assigned_to": "alice",\n  "status": "New",\n  "created_at": "2024-10-28T00:30:52.822710",\n  "updated_at": "2024-10-28T00:30:52.822710",\n  "estimated_completion": "2024-11-04T00:30:52.822710"\n}'},
 {'content': 'La tarea 1, titulada "Corrección de bug crítico en la página de login", actualmente está en estado **New**. Esta ta